In [98]:
import pickle
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)
#pip install pandas-profiling

In [99]:
# df_review = pd.read_pickle('reviews_all.pkl')
# df_review

In [100]:
df_main = pd.read_pickle('df_main.pkl')
df_main

,prod_name,rating,rating_count,desc,profile,vitolas,Binder,Filler,Flavored,Has Tip,Origin,Pressed,Shapes,Sweet,Wrapper,Brand:
0,1876 Reserve,4.3,513,Even gramps would be proud.On a trip to factor...,Mellow,"[Churchill, Robusto, Toro, Torpedo]",Sumatra,Dominican,False,No,Dominican Republic,No,"Churchill, Robusto, Toro, Torpedo",No,Connecticut,NaN
1,1876 Reserve Maduro,4.19,247,A dark horse from 1876 Reserve.Maestro....drum...,Mellow-Medium,"[Churchill, Robusto, Toro, Torpedo]",NaN,Dominican,False,No,Dominican Republic,No,"Churchill, Robusto, Toro, Torpedo",No,Maduro,NaN
2,3x3 Tubos by Davidoff,4.44,16,3x3 = an Unbridled Winner! There’s tons of cig...,Mellow-Medium,"[Churchill, Corona, Robusto]",Dominican,Dominican,False,No,Dominican Republic,No,"Churchill, Corona, Robusto",No,Connecticut,Davidoff
3,5 Vegas Big Five,4.22,64,High five! 5 Vegas's first new release in year...,Full,"[Churchill, Gordo, Torpedo]",Nicaraguan,Nicaraguan,False,No,Nicaragua,No,"Churchill, Gordo, Torpedo",Varies,"Habano, Nicaraguan",5 Vegas
4,5 Vegas Cask-Strength,4.5,101,A luxurious encore presentation.The term ’cask...,Medium-Full,"[Gordo, Toro]",Nicaraguan,"Honduran, Nicaraguan",False,No,Nicaragua,Yes,"Gordo, Toro",No,Corojo,5 Vegas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1611,White Owl Cigars,4,15,White Owl cigars are something to hoot about!W...,Mellow,"[Corona, Lancero/Panatela]",NaN,NaN,False,No,United States,No,"Corona, Lancero/Panatela",Yes,HTL,NaN
1612,Wilde Cigars,4.24,59,Wild thing! You make my heart sing!Excuse my l...,Mellow,[Lancero/Panatela],NaN,Dominican,False,No,Dominican Republic,No,Lancero/Panatela,No,Connecticut,NaN
1613,Wynwood Hills,4.5,2,"Back again, and completely “Unhinged” Original...",Medium-Full,[Rothschild],NaN,NaN,False,No,Honduras,No,Rothschild,No,"Connecticut Broadleaf, Corojo, Sumatra",NaN
1614,Zino Platinum Crown Series,5,2,"Strong, powerful, flavorful....Crown Series hi...",Medium-Full,"[Churchill, Gordo, Toro]",Connecticut,"Dominican, Peruvian",False,No,Dominican Republic,No,"Churchill, Gordo, Toro",No,Ecuador Connecticut,NaN


In [104]:
from sklearn.preprocessing import MultiLabelBinarizer
 

s = df_main['vitolas']

mlb = MultiLabelBinarizer()

df_vitola = pd.DataFrame(mlb.fit_transform(s),columns= (f'vitola_' + mlb.classes_), index=df_main.index)

In [105]:
pd.concat([df_main, df_vitola], axis=1)

,prod_name,rating,rating_count,desc,profile,vitolas,Binder,Filler,Flavored,Has Tip,Origin,Pressed,Shapes,Sweet,Wrapper,Brand:,vitola_Belicoso,vitola_Churchill,vitola_Cigarillos,vitola_Corona,vitola_Corona Especial,vitola_Corona Extra,vitola_Corona Gigante,vitola_Corona Gordo,vitola_Corona Grande,vitola_Culebra,vitola_Double Corona,vitola_Double Perfecto,vitola_Double Robusto,vitola_Double Toro,vitola_Figurado,vitola_Gigante,vitola_Gordo,vitola_Gordo Extra,vitola_Gran Rothschild,vitola_Lancero,vitola_Lancero/Panatela,vitola_Lancero/Pantella,vitola_Lonsdale,vitola_Panatela,vitola_Perfecto,vitola_Petite Corona,vitola_Presidente,vitola_Pyramid,vitola_Robusto,vitola_Robusto Extra,vitola_Rothschild,vitola_Salomon,vitola_Short Robusto,vitola_Toro,vitola_Torpedo,vitola_Wedge
0,1876 Reserve,4.3,513,Even gramps would be proud.On a trip to factor...,Mellow,"[Churchill, Robusto, Toro, Torpedo]",Sumatra,Dominican,False,No,Dominican Republic,No,"Churchill, Robusto, Toro, Torpedo",No,Connecticut,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0
1,1876 Reserve Maduro,4.19,247,A dark horse from 1876 Reserve.Maestro....drum...,Mellow-Medium,"[Churchill, Robusto, Toro, Torpedo]",NaN,Dominican,False,No,Dominican Republic,No,"Churchill, Robusto, Toro, Torpedo",No,Maduro,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0
2,3x3 Tubos by Davidoff,4.44,16,3x3 = an Unbridled Winner! There’s tons of cig...,Mellow-Medium,"[Churchill, Corona, Robusto]",Dominican,Dominican,False,No,Dominican Republic,No,"Churchill, Corona, Robusto",No,Connecticut,Davidoff,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,5 Vegas Big Five,4.22,64,High five! 5 Vegas's first new release in year...,Full,"[Churchill, Gordo, Torpedo]",Nicaraguan,Nicaraguan,False,No,Nicaragua,No,"Churchill, Gordo, Torpedo",Varies,"Habano, Nicaraguan",5 Vegas,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,5 Vegas Cask-Strength,4.5,101,A luxurious encore presentation.The term ’cask...,Medium-Full,"[Gordo, Toro]",Nicaraguan,"Honduran, Nicaraguan",False,No,Nicaragua,Yes,"Gordo, Toro",No,Corojo,5 Vegas,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1611,White Owl Cigars,4,15,White Owl cigars are something to hoot about!W...,Mellow,"[Corona, Lancero/Panatela]",NaN,NaN,False,No,United States,No,"Corona, Lancero/Panatela",Yes,HTL,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1612,Wilde Cigars,4.24,59,Wild thing! You make my heart sing!Excuse my l...,Mellow,[Lancero/Panatela],NaN,Dominican,False,No,Dominican Republic,No,Lancero/Panatela,No,Connecticut,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1613,Wynwood Hills,4.5,2,"Back again, and completely “Unhinged” Original...",Medium-Full,[Rothschild],NaN,NaN,False,No,Honduras,No,Rothschild,No,"Connecticut Broadleaf, Corojo, Sumatra",NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1614,Zino Platinum Crown Series,5,2,"Strong, powerful, flavorful....Crown Series hi...",Medium-Full,"[Churchill, Gordo, Toro]",Connecticut,"Dominican, Peruvian",False,No,Dominican Republic,No,"Churchill, Gordo, Toro",No,Ecuador Connecticut,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [106]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1616 entries, 0 to 1615
Data columns (total 16 columns):
prod_name       1616 non-null object
rating          1608 non-null object
rating_count    1394 non-null object
desc            1616 non-null object
profile         1570 non-null object
vitolas         1616 non-null object
Binder          1046 non-null object
Filler          1461 non-null object
Flavored        1616 non-null object
Has Tip         1538 non-null object
Origin          1598 non-null object
Pressed         1615 non-null object
Shapes          1607 non-null object
Sweet           1616 non-null object
Wrapper         1605 non-null object
Brand:          1112 non-null object
dtypes: object(16)
memory usage: 202.1+ KB


In [107]:
df_main.isnull().sum()


prod_name         0
rating            8
rating_count    222
desc              0
profile          46
vitolas           0
Binder          570
Filler          155
Flavored          0
Has Tip          78
Origin           18
Pressed           1
Shapes            9
Sweet             0
Wrapper          11
Brand:          504
dtype: int64

In [108]:
df_main['rating'].fillna( 0, inplace=True)

In [109]:
df_main['rating_count'].fillna( 0, inplace=True)

In [110]:
df_main['Has Tip'].fillna( 'No', inplace=True)

In [111]:
df_main.isnull().sum()

prod_name         0
rating            0
rating_count      0
desc              0
profile          46
vitolas           0
Binder          570
Filler          155
Flavored          0
Has Tip           0
Origin           18
Pressed           1
Shapes            9
Sweet             0
Wrapper          11
Brand:          504
dtype: int64

In [97]:
df1 = df_main[df_main['Shapes'].isnull()]
df1

,prod_name,rating,rating_count,desc,profile,vitolas,Binder,Filler,Flavored,Has Tip,Origin,Pressed,Shapes,Sweet,Wrapper,Brand:
190,Balmoral Anejo XO Connecticut,4.83,6,XO goes to ConnecticutSince they began their r...,Mellow-Medium,[],Sumatra,"Brazilian, Dominican, United States",False,No,Dominican Republic,No,NaN,No,Connecticut,Royal Agio Cigars
192,Balmoral Anejo XO Oscuro,5,0,Go all in on XOBalmoral Anejo has a simple mis...,Medium-Full,[],Dominican,"Brazilian, Dominican, Nicaraguan",False,No,Dominican Republic,No,NaN,No,Dominican,Royal Agio Cigars
287,Camacho Scorpion Fumas Connecticut,4.33,3,"Just a little sting, and your troubles are gon...",Medium,[],NaN,"Honduran, Nicaraguan",False,No,Honduras,No,NaN,No,Ecuador Connecticut,Camacho
618,Four Kicks Mule Kick LE 2019,4.5,0,"The mule is back, with a saddlebag full of fla...",None,[],Nicaraguan,Nicaraguan,False,No,Nicaragua,False,NaN,False,Ecuador Sumatra,NaN
904,La Aurora Bristol Especial Maduro,5,3,A lion's share of hearty flavors for a mouse-s...,None,[],NaN,"Dominican, Nicaraguan",False,No,Dominican Republic,False,NaN,False,Maduro,La Aurora
907,La Aurora Embassador Maduro,4,1,Aged 10 YearsLa Aurora Cigars are steeped in h...,Medium,[],Brazil,"Brazilian, Dominican, Nicaraguan, Peruvian",False,No,Dominican Republic,False,NaN,False,Brazilian,La Aurora
1015,Las 6 Provincias MTZ BP,4,0,A nod to the olde-world.Before Castro rose to ...,None,[],NaN,NaN,False,No,NaN,No,NaN,No,NaN,NaN
1061,Man O' War Valkyrie,5,3,"For a limited time, you'll receive FREE shippi...",Mellow-Medium,[],NaN,NaN,False,No,Nicaragua,No,NaN,No,Connecticut,Man O' War
1133,Nica Libre x AGANORSA,4.57,7,"For a limited time, receive FREE shipping on y...",Medium-Full,[],Nicaraguan,Nicaraguan,False,No,Nicaragua,No,NaN,No,Corojo,Casa Fernandez


In [11]:
# import numpy as np

# from pandas_profiling import ProfileReport

In [12]:
# df_merge['desc'].profile_report()

In [13]:
#pip install pandas-profiling


In [14]:
# df_main['rating'] = [x.split(' ')[1] if x!=None else x for x in df_main['rating']]

In [15]:
# df_main['rating_count'] = [x.split(' ')[3] if x!=None else x for x in df_main['rating_count']]

In [16]:
df_main[["prod_name","desc"]]

,prod_name,desc
0,1876 Reserve,Even gramps would be proud.On a trip to factor...
1,1876 Reserve Maduro,A dark horse from 1876 Reserve.Maestro....drum...
2,3x3 Tubos by Davidoff,3x3 = an Unbridled Winner! There’s tons of cig...
3,5 Vegas Big Five,High five! 5 Vegas's first new release in year...
4,5 Vegas Cask-Strength,A luxurious encore presentation.The term ’cask...
...,...,...
1611,White Owl Cigars,White Owl cigars are something to hoot about!W...
1612,Wilde Cigars,Wild thing! You make my heart sing!Excuse my l...
1613,Wynwood Hills,"Back again, and completely “Unhinged” Original..."
1614,Zino Platinum Crown Series,"Strong, powerful, flavorful....Crown Series hi..."


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = df_main['desc']
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
#print(vectorizer.get_feature_names())

tfid_df = pd.DataFrame(X)

In [26]:
tfid_df

,0
0,"(0, 2766)\t0.08632635780037382\n (0, 6009)\..."
1,"(0, 10728)\t0.09083158165537449\n (0, 2707)..."
2,"(0, 4009)\t0.0518397941698556\n (0, 9432)\t..."
3,"(0, 7747)\t0.08145229428745425\n (0, 6138)\..."
4,"(0, 11012)\t0.045792332635808314\n (0, 1013..."
...,...
1611,"(0, 5643)\t0.10512094525295852\n (0, 8069)\..."
1612,"(0, 7243)\t0.1971576941909345\n (0, 11032)\..."
1613,"(0, 2330)\t0.1401838139849542\n (0, 9888)\t..."
1614,"(0, 9901)\t0.19566664008807405\n (0, 378)\t..."


In [27]:
df_vect = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())

In [28]:
df_vect

,00,000,008,03,042,05,06it,09,10,100,1000,101,1066,107,107th,10s,10th,11,115th,12,120,125,125th,13,130,130th,135th,14,140,1400cc,142,146,146th,1492,1495,15,150,1500,151,1515,15th,16,1666,169,17,170,175,175th,1794,18,1800,1800s,1812,1829,1834,1840,1843,1844,1845,1860,1865,1875,1876,1878,1882,1884,1885,1887,1888,1895,1896,19,1900,1900s,1901,1903,1904,1907,1911,1914,1916,1920,1922,1926,1930,1934,1936,1937,1946,1950,1955,1957,1959,1960,1960s,1961,1962,1963,1964,1966,1968,1969,1970,1970s,1976,1977,1979,1980,1982,1985,1986,1990,1990s,1992,1993,1994,1995,1996,1997,1998,1999,19th,1ra,1st,1sts,20,200,2000,2000s,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,20s,20th,21,2190,21st,22,22nd,23,2306,24,24th,25,2500,25th,28,29,2k,2nd,2nds,30,300,3000,30th,32,320,33,34,375,38,3x3,40,400,4000,401k,40th,411,44,45,46,481,49,4ks,4th,4x66,50,500,500th,505,5050,50th,52,52c,54,543,55,56,574,58,5th,60,600,6000,601,60s,60th,62,64,64ths,65,650,66,67,68,685,6th,6x60,70,700,702,70s,70th,7100,743,75,75th,79,7th,7x54,80,800,80th,826,84,843,85,858,89,8x80,90,90s,90th,...,when,whenever,where,wherever,whether,which,whichever,while,whim,whipped,whirl,whiskey,whiskies,whisky,whisperer,whispers,whistles,white,whiter,whitman,who,whoa,whoever,whole,wholly,whom,whose,why,wick,wicked,wide,widely,widen,wider,widow,width,wife,wik,wikipedia,wild,wilde,wildfire,wildly,will,william,willing,willpower,willy,win,wind,winding,window,wine,winemakers,wines,wing,wink,winkle,winner,winnerbahia,winners,winnerthe,winning,wins,winston,winter,winterman,wintermans,wipe,wisdom,wise,wisely,wisest,wish,wishes,wishing,witch,with,within,without,withstand,withstood,witness,witnessed,wizardry,wo,wolf,women,won,wonder,wonderful,wonderfully,wondering,wonders,wondrous,wont,wood,wooden,woodiness,woodlawn,woods,woodsouthern,woodsy,woodwork,woody,word,words,wore,work,workbench,worked,worker,workers,working,workload,works,workshop,workshopwhat,world,worldly,worldover,worldreleased,worlds,worldwide,worn,worries,worry,worse,worship,worst,worth,worthwhile,worthy,would,wouldn,wound,wounds,wow,wowed,wrangle,wrangles,wrap,wrapp,wrapped,wrapper,wrappernegra,wrappers,wrappersmany,wrapping,wrappping,wraps,wreak,wrecking,wright,write,writers,writing,written,wrong,wrongs,wrote,wset,wtf,wwii,wynne,wynwood,x39,x42,x44,x48,x50,x52,x54,x56,x58,x60,x66,x70,xiii,ximenez,xo,xobalmoral,xt,xtreme,xviii,xxi,ya,yakuza,yamasa,yamasay,yamasuma,yang,yankee,yard,yardwork,yawn,ybor,ye,yea,yeah,year,yearly,yearn,years,yearsla,yelled,yellow,yellowish,yep,yer,yes,yesteryear,yet,yield,yielded,yielding,yields,yin,ynclan,ynlcan,yo,yogi,yonder,yore,york,you,young,younger,your,yours,yourself,yourselfah,yourselves,youwhen,yr,yugo,yummy,yup,zambia,zeal,zeppelin,zero,zest,zesty,zing,zino,zodiak,zona,zoning,zulu
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.134674,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [34]:
# 2. only keep words w average tfidf scores over 0.0001
relevant = []
for word in df_vect.columns:
    if df_vect[word].mean() > 0.0001:
        relevant.append(df_vect[word])
len(relevant)

7419

In [20]:
frames = [df_main, df_vect]

result = pd.concat(frames, axis=1)
result

,prod_name,rating,rating_count,desc,profile,vitolas,Binder,Filler,Flavored,Has Tip,Origin,Pressed,Shapes,Sweet,Wrapper,Brand:,00,000,008,03,042,05,06it,09,10,100,1000,101,1066,107,107th,10s,10th,11,115th,12,120,125,125th,13,130,130th,135th,14,140,1400cc,142,146,146th,1492,1495,15,150,1500,151,1515,15th,16,1666,169,17,170,175,175th,1794,18,1800,1800s,1812,1829,1834,1840,1843,1844,1845,1860,1865,1875,1876,1878,1882,1884,1885,1887,1888,1895,1896,19,1900,1900s,1901,1903,1904,1907,1911,1914,1916,1920,1922,1926,1930,1934,1936,1937,1946,1950,1955,1957,1959,1960,1960s,1961,1962,1963,1964,1966,1968,1969,1970,1970s,1976,1977,1979,1980,1982,1985,1986,1990,1990s,1992,1993,1994,1995,1996,1997,1998,1999,19th,1ra,1st,1sts,20,200,2000,2000s,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,20s,20th,21,2190,21st,22,22nd,23,2306,24,24th,25,2500,25th,28,29,2k,2nd,2nds,30,300,3000,30th,32,320,33,34,375,38,3x3,40,400,4000,401k,40th,411,44,45,46,481,49,4ks,4th,4x66,50,500,500th,505,5050,50th,52,52c,54,543,55,56,574,58,5th,60,600,6000,601,60s,60th,62,64,64ths,65,650,66,67,68,685,6th,6x60,70,700,702,70s,70th,7100,743,75,75th,...,when,whenever,where,wherever,whether,which,whichever,while,whim,whipped,whirl,whiskey,whiskies,whisky,whisperer,whispers,whistles,white,whiter,whitman,who,whoa,whoever,whole,wholly,whom,whose,why,wick,wicked,wide,widely,widen,wider,widow,width,wife,wik,wikipedia,wild,wilde,wildfire,wildly,will,william,willing,willpower,willy,win,wind,winding,window,wine,winemakers,wines,wing,wink,winkle,winner,winnerbahia,winners,winnerthe,winning,wins,winston,winter,winterman,wintermans,wipe,wisdom,wise,wisely,wisest,wish,wishes,wishing,witch,with,within,without,withstand,withstood,witness,witnessed,wizardry,wo,wolf,women,won,wonder,wonderful,wonderfully,wondering,wonders,wondrous,wont,wood,wooden,woodiness,woodlawn,woods,woodsouthern,woodsy,woodwork,woody,word,words,wore,work,workbench,worked,worker,workers,working,workload,works,workshop,workshopwhat,world,worldly,worldover,worldreleased,worlds,worldwide,worn,worries,worry,worse,worship,worst,worth,worthwhile,worthy,would,wouldn,wound,wounds,wow,wowed,wrangle,wrangles,wrap,wrapp,wrapped,wrapper,wrappernegra,wrappers,wrappersmany,wrapping,wrappping,wraps,wreak,wrecking,wright,write,writers,writing,written,wrong,wrongs,wrote,wset,wtf,wwii,wynne,wynwood,x39,x42,x44,x48,x50,x52,x54,x56,x58,x60,x66,x70,xiii,ximenez,xo,xobalmoral,xt,xtreme,xviii,xxi,ya,yakuza,yamasa,yamasay,yamasuma,yang,yankee,yard,yardwork,yawn,ybor,ye,yea,yeah,year,yearly,yearn,years,yearsla,yelled,yellow,yellowish,yep,yer,yes,yesteryear,yet,yield,yielded,yielding,yields,yin,ynclan,ynlcan,yo,yogi,yonder,yore,york,you,young,younger,your,yours,yourself,yourselfah,yourselves,youwhen,yr,yugo,yummy,yup,zambia,zeal,zeppelin,zero,zest,zesty,zing,zino,zodiak,zona,zoning,zulu
0,1876 Reserve,4.3,513,Even gramps would be proud.On a trip to factor...,Mellow,"[Churchill, Robusto, Toro, Torpedo]",Sumatra,Dominican,False,No,Dominican Republic,No,"Churchill, Robusto, Toro, Torpedo",No,Connecticut,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.134674,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [ ]:
df_main["desc"]= df_main["desc"].astype(str) 


In [ ]:
df_main.isna().sum()

In [ ]:

wordcloud = WordCloud(
                          collocations = False,
                          width=1600, height=800,
                          background_color='white',
                          stopwords=stopwords,
                          max_words=150,
                          #max_font_size=40, 
                          random_state=42
                         ).generate(' '.join(df_main['desc'])) # can't pass a series, needs to be strings and function computes frequencies
print(wordcloud)
plt.figure(figsize=(9,8))
fig = plt.figure(1)
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
stemmer = PorterStemmer()
def stemming_tokenizer(str_input):
    words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split()
    words = [porter_stemmer.stem(word) for word in words]
    return words

In [ ]:

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
data = df_main['desc']
# data.head()

tf_idf_vectorizor = TfidfVectorizer(stop_words = stopwords,#tokenizer = tokenize_and_stem,
                             max_features = 5000)
%time tf_idf = tf_idf_vectorizor.fit_transform(data)
tf_idf_norm = normalize(tf_idf)
tf_idf_array = tf_idf_norm.toarray()
pd.DataFrame(tf_idf_array, columns=tf_idf_vectorizor.get_feature_names()).head()

In [ ]:
initial_centroids = np.random.permutation(tf_idf_array.shape[0])[:3]
initial_centroids
centroids = tf_idf_array[initial_centroids]
centroids.shape
dist_to_centroid =  pairwise_distances(tf_idf_array,centroids, metric = 'euclidean')
cluster_labels = np.argmin(dist_to_centroid, axis = 1)

In [ ]:
class Kmeans:
    """ K Means Clustering
    
    Parameters
    -----------
        k: int , number of clusters
        
        seed: int, will be randomly set if None
        
        max_iter: int, number of iterations to run algorithm, default: 200
        
    Attributes
    -----------
       centroids: array, k, number_features
       
       cluster_labels: label for each data point
       
    """
    
    def __init__(self, k, seed = None, max_iter = 200):
        self.k = k
        self.seed = seed
        if self.seed is not None:
            np.random.seed(self.seed)
        self.max_iter = max_iter
        
            
    
    def initialise_centroids(self, data):
        """Randomly Initialise Centroids
        
        Parameters
        ----------
        data: array or matrix, number_rows, number_features
        
        Returns
        --------
        centroids: array of k centroids chosen as random data points 
        """
        
        initial_centroids = np.random.permutation(data.shape[0])[:self.k]
        self.centroids = data[initial_centroids]

        return self.centroids
    
    
    def assign_clusters(self, data):
        """Compute distance of data from clusters and assign data point
           to closest cluster.
        
        Parameters
        ----------
        data: array or matrix, number_rows, number_features
        
        Returns
        --------
        cluster_labels: index which minmises the distance of data to each
        cluster
            
        """
        
        if data.ndim == 1:
            data = data.reshape(-1, 1)
        
        dist_to_centroid =  pairwise_distances(data, self.centroids, metric = 'euclidean')
        self.cluster_labels = np.argmin(dist_to_centroid, axis = 1)
        
        return  self.cluster_labels
    
    
    def update_centroids(self, data):
        """Computes average of all data points in cluster and
           assigns new centroids as average of data points
        
        Parameters
        -----------
        data: array or matrix, number_rows, number_features
        
        Returns
        -----------
        centroids: array, k, number_features
        """
        
        self.centroids = np.array([data[self.cluster_labels == i].mean(axis = 0) for i in range(self.k)])
        
        return self.centroids
    
    
    def convergence_calculation(self):
        """
        Calculates 
        
        """
        pass
    
    def predict(self, data):
        """Predict which cluster data point belongs to
        
        Parameters
        ----------
        data: array or matrix, number_rows, number_features
        
        Returns
        --------
        cluster_labels: index which minmises the distance of data to each
        cluster
        """
        
        return self.assign_clusters(data)
    
    def fit_kmeans(self, data):
        """
        This function contains the main loop to fit the algorithm
        Implements initialise centroids and update_centroids
        according to max_iter
        -----------------------
        
        Returns
        -------
        instance of kmeans class
            
        """
        self.centroids = self.initialise_centroids(data)
        
        # Main kmeans loop
        for iter in range(self.max_iter):

            self.cluster_labels = self.assign_clusters(data)
            self.centroids = self.update_centroids(data)          
            if iter % 100 == 0:
                print("Running Model Iteration %d " %iter)
        print("Model finished running")
        return self

In [ ]:
from sklearn.datasets import make_blobs
# create blobs
data = make_blobs(n_samples=200, n_features=2, centers=4, cluster_std=1.6, random_state=50)
# create np array for data points
points = data[0]
# create scatter plot
plt.scatter(data[0][:,0], data[0][:,1], c=data[1], cmap='viridis')
plt.xlim(-15,15)
plt.ylim(-15,15)

X = data[0]
X[2]

In [ ]:
df_main.info()

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.cluster import KMeans 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from scipy.stats import multivariate_normal as mvn
import nltk
import os
import random




import string
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# email module has some useful functions
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')